##### Copyright 2018 Die TensorFlow-Autoren.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Trainingskontrollpunkte

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ansicht auf TensorFlow.org</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Führen Sie es in Google Colab aus</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Quelle auf GitHub ansehen</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Notizbuch herunterladen</a>   </td>
</table>

Der Ausdruck „Speichern eines TensorFlow-Modells“ bedeutet normalerweise eines von zwei Dingen:

1. Kontrollpunkte, OR
2. Gespeichertes Modell.

Prüfpunkte erfassen den genauen Wert aller von einem Modell verwendeten Parameter ( `tf.Variable` -Objekte). Prüfpunkte enthalten keine Beschreibung der vom Modell definierten Berechnung und sind daher normalerweise nur dann nützlich, wenn Quellcode verfügbar ist, der die gespeicherten Parameterwerte verwendet.

Das SavedModel-Format hingegen enthält zusätzlich zu den Parameterwerten (Checkpoint) eine serialisierte Beschreibung der durch das Modell definierten Berechnung. Modelle in diesem Format sind unabhängig vom Quellcode, der das Modell erstellt hat. Sie eignen sich daher für die Bereitstellung über TensorFlow Serving, TensorFlow Lite, TensorFlow.js oder Programme in anderen Programmiersprachen (die TensorFlow-APIs C, C++, Java, Go, Rust, C# usw.).

In diesem Leitfaden werden APIs zum Schreiben und Lesen von Prüfpunkten behandelt.

In [0]:
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

## Speichern von `tf.keras` Trainings-APIs

[Weitere Informationen zum Speichern und Wiederherstellen finden Sie im `tf.keras` Handbuch](./keras/overview.ipynb#save_and_restore) .

`tf.keras.Model.save_weights` speichert einen TensorFlow-Prüfpunkt. 

In [0]:
net.save_weights('easy_checkpoint')

## Checkpoints schreiben


Der persistente Zustand eines TensorFlow-Modells wird in `tf.Variable` -Objekten gespeichert. Diese können direkt erstellt werden, werden jedoch häufig über High-Level-APIs wie `tf.keras.layers` oder `tf.keras.Model` erstellt.

Der einfachste Weg, Variablen zu verwalten, besteht darin, sie an Python-Objekte anzuhängen und dann auf diese Objekte zu verweisen.

Unterklassen von `tf.train.Checkpoint` , `tf.keras.layers.Layer` und `tf.keras.Model` verfolgen automatisch Variablen, die ihren Attributen zugewiesen sind. Das folgende Beispiel erstellt ein einfaches lineares Modell und schreibt dann Prüfpunkte, die Werte für alle Variablen des Modells enthalten.

Mit `Model.save_weights` können Sie ganz einfach einen Modellprüfpunkt speichern

### Manuelles Checkpointing

#### Aufstellen

Um alle Funktionen von `tf.train.Checkpoint` zu demonstrieren, definieren Sie einen Spielzeugdatensatz und einen Optimierungsschritt:

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### Erstellen Sie die Prüfpunktobjekte

Um manuell einen Prüfpunkt zu erstellen, benötigen Sie ein `tf.train.Checkpoint` Objekt. Hier werden die Objekte, die Sie überprüfen möchten, als Attribute für das Objekt festgelegt.

Ein `tf.train.CheckpointManager` kann auch bei der Verwaltung mehrerer Checkpoints hilfreich sein.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### Trainieren und überprüfen Sie das Modell

Die folgende Trainingsschleife erstellt eine Instanz des Modells und eines Optimierers und fasst sie dann in einem `tf.train.Checkpoint` Objekt zusammen. Es ruft den Trainingsschritt in einer Schleife für jeden Datenstapel auf und schreibt regelmäßig Prüfpunkte auf die Festplatte.

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

#### Stellen Sie das Training wieder her und setzen Sie es fort

Nach dem ersten können Sie ein neues Model und einen neuen Manager bestehen, aber das Training genau dort fortsetzen, wo Sie aufgehört haben:

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

Das `tf.train.CheckpointManager` Objekt löscht alte Prüfpunkte. Oben ist es so konfiguriert, dass nur die drei letzten Prüfpunkte beibehalten werden.

In [0]:
print(manager.checkpoints)  # List the three remaining checkpoints

Diese Pfade, z. B. `'./tf_ckpts/ckpt-10'` , sind keine Dateien auf der Festplatte. Stattdessen sind sie Präfixe für eine `index` und eine oder mehrere Datendateien, die die Variablenwerte enthalten. Diese Präfixe werden in einer einzigen `checkpoint` ( `'./tf_ckpts/checkpoint'` ) gruppiert, in der der `CheckpointManager` seinen Status speichert.

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## Lademechanik

TensorFlow ordnet Variablen Prüfpunktwerten zu, indem es ausgehend vom geladenen Objekt einen gerichteten Graphen mit benannten Kanten durchläuft. Kantennamen stammen normalerweise aus Attributnamen in Objekten, zum Beispiel `"l1"` in `self.l1 = tf.keras.layers.Dense(5)` . `tf.train.Checkpoint` verwendet seine Schlüsselwortargumentnamen, wie im `"step"` in `tf.train.Checkpoint(step=...)` .

Das Abhängigkeitsdiagramm aus dem obigen Beispiel sieht folgendermaßen aus:

![Visualisierung des Abhängigkeitsdiagramms für die Beispiel-Trainingsschleife](https://tensorflow.org/images/guide/whole_checkpoint.svg)

Mit dem Optimierer in Rot, regulären Variablen in Blau und Optimierungs-Slot-Variablen in Orange. Die anderen Knoten, die beispielsweise den `tf.train.Checkpoint` darstellen, sind schwarz.

Slot-Variablen sind Teil des Status des Optimierers, werden jedoch für eine bestimmte Variable erstellt. Beispielsweise entsprechen die `'m'` -Kanten oben dem Impuls, den der Adam-Optimierer für jede Variable verfolgt. Slot-Variablen werden nur dann in einem Prüfpunkt gespeichert, wenn sowohl die Variable als auch der Optimierer gespeichert würden, daher die gestrichelten Kanten.

Der Aufruf von `restore()` für ein `tf.train.Checkpoint` Objekt stellt die angeforderten Wiederherstellungen in die Warteschlange und stellt Variablenwerte wieder her, sobald es einen passenden Pfad vom `Checkpoint` Objekt gibt. Beispielsweise können wir nur den Bias aus dem oben definierten Modell laden, indem wir einen Pfad dorthin durch das Netzwerk und die Schicht rekonstruieren.

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

Der Abhängigkeitsgraph für diese neuen Objekte ist ein viel kleinerer Teilgraph des größeren Prüfpunkts, den wir oben beschrieben haben. Es enthält nur die Vorspannung und einen Speicherzähler, den `tf.train.Checkpoint` zum Nummerieren von Prüfpunkten verwendet.

![Visualisierung eines Untergraphen für die Bias-Variable](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()` gibt ein Statusobjekt zurück, das optionale Zusicherungen enthält. Alle Objekte, die wir in unserem neuen `Checkpoint` erstellt haben, wurden wiederhergestellt, sodass `status.assert_existing_objects_matched()` erfolgreich ist.

In [0]:
status.assert_existing_objects_matched()

Es gibt viele Objekte im Prüfpunkt, die nicht übereinstimmen, einschließlich des Kernels der Ebene und der Variablen des Optimierers. `status.assert_consumed()` besteht nur, wenn der Prüfpunkt und das Programm genau übereinstimmen, und würde hier eine Ausnahme auslösen.

### Verzögerte Restaurierungen

`Layer` Objekte in TensorFlow können die Erstellung von Variablen bis zu ihrem ersten Aufruf verzögern, wenn Eingabeformen verfügbar sind. Beispielsweise hängt die Form des Kernels einer `Dense` Ebene sowohl von der Eingabe- als auch von der Ausgabeform der Ebene ab. Daher reicht die als Konstruktorargument erforderliche Ausgabeform nicht aus, um die Variable selbst zu erstellen. Da beim Aufrufen eines `Layer` auch der Wert der Variablen gelesen wird, muss zwischen der Erstellung der Variablen und ihrer ersten Verwendung eine Wiederherstellung erfolgen.

Um diese Redewendung zu unterstützen, stellt `tf.train.Checkpoint` Wiederherstellungen in die Warteschlange, für die noch keine passende Variable vorhanden ist.

In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### Kontrollpunkte manuell überprüfen

`tf.train.list_variables` listet die Prüfpunktschlüssel und Formen von Variablen in einem Prüfpunkt auf. Prüfpunktschlüssel sind Pfade im oben angezeigten Diagramm.

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### Listen- und Wörterbuchverfolgung

Wie bei direkten Attributzuweisungen wie `self.l1 = tf.keras.layers.Dense(5)` wird durch die Zuweisung von Listen und Wörterbüchern zu Attributen deren Inhalt verfolgt.

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

Möglicherweise bemerken Sie Wrapper-Objekte für Listen und Wörterbücher. Diese Wrapper sind checkpointbare Versionen der zugrunde liegenden Datenstrukturen. Genau wie das attributbasierte Laden stellen diese Wrapper den Wert einer Variablen wieder her, sobald sie dem Container hinzugefügt wird.

In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

Die gleiche Nachverfolgung wird automatisch auf Unterklassen von `tf.keras.Model` angewendet und kann beispielsweise zum Nachverfolgen von Ebenenlisten verwendet werden.

## Speichern objektbasierter Prüfpunkte mit Estimator

Weitere Informationen finden Sie im [Estimator-Leitfaden](https://www.tensorflow.org/guide/estimator) .

Schätzer speichern Prüfpunkte standardmäßig mit Variablennamen und nicht mit dem in den vorherigen Abschnitten beschriebenen Objektdiagramm. `tf.train.Checkpoint` akzeptiert namensbasierte Prüfpunkte, aber Variablennamen können sich ändern, wenn Teile eines Modells außerhalb des `model_fn` des Schätzers verschoben werden. Durch das Speichern objektbasierter Prüfpunkte ist es einfacher, ein Modell innerhalb eines Schätzers zu trainieren und es dann außerhalb eines Schätzers zu verwenden.

In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint` kann dann die Prüfpunkte des Schätzers aus seinem `model_dir` laden.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## Zusammenfassung

TensorFlow-Objekte bieten einen einfachen automatischen Mechanismus zum Speichern und Wiederherstellen der Werte der von ihnen verwendeten Variablen.
